# Está hecho para ejecutarse en Windows, con el chromedriver.exe

In [100]:
import undetected_chromedriver as uc
import pandas as pd
import random
import time
from selenium import webdriver
from selenium.webdriver.support.ui import Select
# import By from selenium
from selenium.webdriver.common.by import By
# from selenium.webdriver.common import By
# from selenium.webdriver.keys import Keys
from bs4 import BeautifulSoup as bs
import requests

In [101]:
COOKIES_XPATH = '//*[@id="cookiesjsr"]/div/div/div[2]/button[3]'
SECTOR_XPATH = '//*[@id="edit-type"]'
GENDER_XPATH = '//*[@id="edit-gender"]'
EVENT_XPATH = '//*[@id="edit-event"]'
CATEGORY_XPATH = '//*[@id="edit-category"]'

In [115]:
def get_select_options(driver, xpath, default_value=False):
    select_element = driver.find_element(By.XPATH, xpath)
    select = Select(select_element)
    return select
    # if default_value:
    #     return select.options[1:]
    # return select.options

In [103]:
def select_category(driver, index):
    category_select = driver.find_element(By.XPATH, CATEGORY_XPATH)
    category_select = Select(category_select)
    option_list = category_select.options
    option_list[index].click()

In [104]:

def click_show(driver):
    show_ranking_button = driver.find_element(By.XPATH, '//*[@id="edit-button"]')
    show_ranking_button.click()

In [105]:
def click_all_marks(driver):
    top_n = driver.find_element(By.XPATH, '//*[@id="ranking_container"]/div[1]/div/select/option[3]')
    top_n.click()

In [106]:
def accept_cookies(driver):
    cookies_button = driver.find_element(By.XPATH, COOKIES_XPATH)
    cookies_button.click()

In [31]:
browser = uc.Chrome()

In [32]:
browser = uc.Chrome()
url = 'https://www.rfearanking.es/ranking'
browser.get(url)
browser.implicitly_wait(10)
time.sleep(10)
accept_cookies = browser.find_element(By.XPATH, COOKIES_XPATH)
accept_cookies.click()

In [33]:
accept_cookies = browser.find_element(By.XPATH, COOKIES_XPATH)
accept_cookies.click()

In [7]:
genre = browser.find_element('xpath', genres['male'])
genre.click()

In [16]:
# category = browser.find_element('xpath', '//*[@id="edit-category"]/option[3]')
# category.click()

In [17]:
event_xpath = '//*[@id="edit-event"]/option[2]'

event = browser.find_element('xpath', event_xpath)
event.click()


In [20]:
html = browser.page_source

In [21]:
soup = bs(html, 'html.parser')

In [26]:
table = soup.find('table', {'class': 'tabla_ranking'})

In [25]:

df = pd.read_html(str(table))[0]
df.head()

,RANK,MARCA,ATLETA,CLUB,F.N.,PAÍS,FED.,POS.,CIUDAD,FECHA,Unnamed: 10
0,1,3:39.04,RONALDO OLIVO MERCEDES,Playas de Castellon,08/10/2004,ESP,MAD,13C1,Castellón,14/06/2023,show
1,2,3:46.64,RUBEN EGEA VIDAL,Alcampo Scorpio 71,28/06/2004,ESP,ARA,7,Huelva,06/06/2023,show
2,3,3:47.28,ALEIX VIVES ESPLUGAS,FACVAC Valls,27/09/2004,ESP,CAT,3C1,Barcelona-SE,21/06/2023,show
3,4,3:47.76,ALEX PINTADO AZNAR,UA Terrassa,13/10/2006,ESP,CAT,8C1,Mataró,02/06/2023,show
4,5,3:47.89,AITOR GARCIA CALVO,Atletismo Alcorcon,15/03/2004,ESP,MAD,4C3,Barcelona,05/07/2023,show


In [31]:
df.to_parquet('ranking.parquet')

In [68]:
# Select options
category_select = get_select_options(browser, CATEGORY_XPATH)
sector_select = get_select_options(browser, SECTOR_XPATH)
gender_select = get_select_options(browser, GENDER_XPATH)
event_select = get_select_options(browser, EVENT_XPATH)


In [107]:
browser = uc.Chrome()
url = 'https://www.rfearanking.es/ranking'
browser.get(url)
browser.implicitly_wait(10)

In [108]:
accept_cookies(browser)

In [119]:
# Select options
gender_select = get_select_options(browser, GENDER_XPATH, False)
gender_names = ['Hombre', 'Mujer']
category_select = get_select_options(browser, CATEGORY_XPATH)

category_index = 3

for gender in gender_names:
    gender_select.select_by_visible_text(gender)
    
    time.sleep(2)
    category_select.select_by_index(category_index)
    time.sleep(2)
    sector_select = get_select_options(browser, SECTOR_XPATH, True)
    sector_names = [sector.accessible_name for sector in sector_select.options[1:]]

    for sector in sector_names:
        sector_select.select_by_visible_text(sector)
        time.sleep(3)

        event_select = get_select_options(browser, EVENT_XPATH, True)
        event_names = [event.accessible_name for event in event_select.options[1:]]
        for event in event_names:
            event_select.select_by_visible_text(event)
            time.sleep(2)

            click_show(browser)
            time.sleep(5)
            try:
                click_all_marks(browser)
            except:
                continue
            time.sleep(5)

            html = browser.page_source
            soup = bs(html, 'html.parser')
            table = soup.find('table', {'class': 'tabla_ranking'})

            df = pd.read_html(str(table))[0]
            df['PRUEBA'] = event
            df['SECTOR'] = sector

        
            file_name = f'ranking_{event}_{gender}.parquet'
            print(df.shape, file_name)
            df.to_parquet(file_name)
            time.sleep(1)

(100, 14) ranking_100m_Hombre.parquet
(100, 14) ranking_200m_Hombre.parquet
(100, 13) ranking_400m_Hombre.parquet
(100, 13) ranking_1.500m_Hombre.parquet
(100, 13) ranking_800m_Hombre.parquet
(100, 13) ranking_3.000m_Hombre.parquet
(91, 13) ranking_5.000m_Hombre.parquet
(98, 13) ranking_2.000m Obst. 0,838m_Hombre.parquet
(100, 14) ranking_110m.v. (0,914)_Hombre.parquet
(93, 13) ranking_400m.v. (0,84)_Hombre.parquet
(100, 13) ranking_Altura_Hombre.parquet
(100, 13) ranking_Pértiga_Hombre.parquet
(100, 14) ranking_Longitud_Hombre.parquet
(100, 14) ranking_Triple_Hombre.parquet
(100, 13) ranking_Peso (3kg)_Hombre.parquet
(100, 13) ranking_Peso (5kg)_Hombre.parquet
(100, 13) ranking_Disco (1,5kg)_Hombre.parquet
(49, 13) ranking_Martillo (5kg)_Hombre.parquet
(100, 13) ranking_Jabalina (700g)_Hombre.parquet
(77, 13) ranking_Decatlón Sub18_Hombre.parquet
(100, 13) ranking_5.000m Marcha_Hombre.parquet
(25, 13) ranking_10.000m Marcha_Hombre.parquet
(24, 13) ranking_4x100m_Hombre.parquet
(100, 1